In [ ]:
import sys
sys.path.append("../libs/")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import shift_join_data, print_hp
from models import build_mlp
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import HyperModel
import datetime
import keras_tuner as kt

# Carga de datos

In [ ]:
df = pd.read_csv('../data/wb_dataset_prep.csv')
df = df.drop('country',axis=1)
iso = df['iso'].unique()    #Codigos de paises
df = df.set_index(['iso','year'])

# Dividir Datos

In [ ]:
iso_test = ['PER']
iso_train = iso[(iso != iso_test[0])]
target_col = ['rgdp_growth']
features = df.columns[(df.columns!=target_col[0])]

# Normalizar

In [ ]:
df_x_train = df.loc[iso_train][features].copy()
df_y_train = df.loc[iso_train][target_col].copy()

std_scaler_x_train = StandardScaler()
std_scaler_y_train = StandardScaler()

df_x_train.iloc[:,:] = std_scaler_x_train.fit_transform(df_x_train)
df_y_train.iloc[:,:] = std_scaler_y_train.fit_transform(df_y_train)

df_x_train.iloc[:,:] = np.clip(df_x_train,-5,5)
df_y_train.iloc[:,:] = np.clip(df_y_train,-5,5)

# Optimizacion

In [ ]:
class MLPHyperModel(HyperModel):

    def __init__(self, n_features_in, n_steps_out,countries ,metrics = ['mae'], name=None, tunable=True):
        super().__init__(name=name, tunable=tunable)
        self.n_features = n_features_in
        self.n_steps_out = n_steps_out
        self.countries = countries
        self.metrics = metrics

    def build(self, hp):
        # Parametrizamos nro de capas, nro de nodos y ratio de aprendizaje
        hp_time_steps = hp.Int('steps_in',4,10,step=1)
        hp_capas = hp.Int('nro_capas',1,5)
        hp_nodes = hp.Int('nro_nodos',32,356,step=16)
        hp_ratio_aprendizaje = hp.Choice('ratio_aprendizaje', values=[1e-2, 1e-3, 1e-4])
        hp_activation = hp.Choice('activacion', values=['relu','tanh'])

        return build_mlp(
            n_steps_in = hp_time_steps,
            n_features = self.n_features,
            n_steps_out = self.n_steps_out, 
            nodes = hp_nodes,
            layers = hp_capas,
            learning_rate = hp_ratio_aprendizaje,
            activation = hp_activation,
            metrics = self.metrics)

    def fit(self, hp, model,x,y,**kwargs):
        x_s,y_s = shift_join_data(x, y, self.countries, hp.get('steps_in'), self.n_steps_out)
        n_batch = len(x_s)
        return model.fit(x = x_s, y = y_s, batch_size = n_batch, **kwargs)

In [ ]:
n_steps_out = 3
n_features = len(features)
ajuste_path = 'tuning_output/'
fecha_hora = datetime.datetime.now().strftime('%Y%m%d_%H%M')
objective = 'val_mae'
max_epochs = 1000
hp_iters = 4

In [ ]:
# Condicion de parada: 50 epocas despues del menor val_loss
#rmse = RootMeanSquaredError(name='rmse')
es = EarlyStopping(monitor='val_mae', mode='min', patience=50)

In [ ]:
mlp_path = 'MLP'+'_'+fecha_hora
mlp_path_prj = mlp_path
mlp_hypermodel = MLPHyperModel(n_features, n_steps_out, iso_train)

mlp_tuner = kt.Hyperband(
    mlp_hypermodel,
    objective = objective,
    max_epochs = max_epochs,
    hyperband_iterations = hp_iters,
    directory = ajuste_path,
    project_name = mlp_path_prj,
    overwrite=True)

mlp_tuner.search(x = df_x_train, y = df_y_train, validation_split = 0.2, epochs = max_epochs
    , verbose = 0, shuffle = False, callbacks = [es])
# guardar mejor modelo
best_mlp_hps = mlp_tuner.get_best_hyperparameters(num_trials = 1)[0]
print_hp(ajuste_path+mlp_path+'.txt',mlp_tuner)

mlp_model = mlp_tuner.hypermodel.build(best_mlp_hps)
mlp_model.save(ajuste_path+mlp_path+'.h5')